In [98]:
import pandas as pd
import numpy as np
from ipywidgets import IntProgress
from matplotlib import pyplot as plt

In [99]:
dif = .1
start = 0.
Fs = [{'label': 'f1', 'F': 1.}]
for i in range(1, 10):
    cur = round(start+dif*i, 2)
    Fs.append({'label':'f'+str(i+1), 'F':cur})

base=.9
dif = .9
for i in range(0, 5):
    dif = dif*.1
    base = base + dif
    cur = round(base, i+2)
    Fs.append({'label':'f'+str(i+11), 'F':cur})

In [95]:
Fs

[{'label': 'f1', 'F': 1.0},
 {'label': 'f2', 'F': 0.1},
 {'label': 'f3', 'F': 0.2},
 {'label': 'f4', 'F': 0.3},
 {'label': 'f5', 'F': 0.4},
 {'label': 'f6', 'F': 0.5},
 {'label': 'f7', 'F': 0.6},
 {'label': 'f8', 'F': 0.7},
 {'label': 'f9', 'F': 0.8},
 {'label': 'f10', 'F': 0.9},
 {'label': 'f11', 'F': 0.99},
 {'label': 'f12', 'F': 0.999},
 {'label': 'f13', 'F': 0.9999},
 {'label': 'f14', 'F': 0.99999},
 {'label': 'f15', 'F': 0.999999}]

In [96]:
chunkMapDF = pd.read_csv('data/ChunkMap.csv')
# qrVals = pd.read_csv('QR_Values.csv')

In [97]:
H_TRANSPOSE=None
H = None

def leastSquare(X, Y, F, SCALED_DATA=False):
    H_TRANSPOSE = np.vstack(([1.]*len(X), X))
    H = np.transpose(H_TRANSPOSE).copy()
    currF = 1.
    i=0
    currF = F
    for x in X:
        if(SCALED_DATA):
            currF = currF*F
        else:
            currF = F**(100-x)
        
        H_TRANSPOSE[0][i], H_TRANSPOSE[1][i] = H_TRANSPOSE[0][i]*currF, H_TRANSPOSE[1][i]*currF
        i+=1
    if(len(H)==1):
        return "HTooShort", "HTooShort"
    else:
        return np.linalg.inv(H_TRANSPOSE @ H) @ H_TRANSPOSE @ Y
# A, B = leastSquare(Xs, np.log(Ys), .9)
# raiseAB(A, B)

def interpXY(X, Y):
    numPoints = pd.Series(X).max()+1
    startX = X[0]
    newX = list(range(startX, numPoints+startX))
    newY = [np.nan]*numPoints    
    for i in range(0, len(X)):
        xV = X[i]-startX
#         print(i, xV)
        newY[xV] = Y[i]
    newY = list(pd.Series(newY).interpolate())
    return newX, newY

def raiseAB(A, B):
    return np.exp(A), B

def genABData(A, B):
#     X = list(range(0, Xf))
    X, Y = [], []
    for i in range(0, 100):
        x = i
        X.append(x)
        Y.append(A*np.exp(B*x))
    return X, Y

def genData(A, B):
#     X = list(range(0, Xf))
    X, Y = [], []
    for i in range(0, 100):
        x = i
        X.append(x)
        Y.append(A+B*x)
    return X, Y

def processRegr(X, Y, F, SCALED_DATA=False):
    A, B = leastSquare(X, np.log(Y), F, SCALED_DATA=SCALED_DATA)
    if(A=="HTooShort"):
        return A, B
    else:
        return raiseAB(A, B)


In [92]:
# plt.plot(X, np.log(Y))
# A, B = leastSquare(X, np.log(Y), .1)
# pred = genData(A, B)
# plt.plot(pred[0], pred[1])

In [205]:
# plt.plot(X, Y)
# A, B = processRegr(X, Y, .1)
# pred = genABData(A, B)
# plt.plot(pred[0], pred[1])

In [184]:
# cSeriesDF.columns

In [90]:
chunkNames = chunkMapDF['Chunk'].unique()[:]

chunkBar = IntProgress(min=0, max=len(chunkNames))
contestBar = IntProgress(min=0, max=10000)

print("Chunks:")
display(chunkBar)

print("Contests:")
display(contestBar)

results = []

for chunkName in chunkNames[:]:
    print(chunkName)
    chunkDF = pd.read_csv('data/Chunks/'+chunkName+'.csv').drop(columns=['Unnamed: 0']).sort_values(by=['MinutesRemaining'], ascending=False)
    
    contests = chunkDF['ContestId'].unique()[:]
    contestBar.value=0
    contestBar.max = len(contests)
    for cid in contests[:]:
#         cSeriesDF = chunkDF[chunkDF['ContestId']==cid]
#         cSeriesDF = cSeriesDF[cSeriesDF['Before4HoursOut']]
#         X = cSeriesDF['TimeScaled']
#         Y = cSeriesDF['EntriesScaled']
#         X, Y = list(X), list(Y)        
#         count = 0

        cSeriesDF = chunkDF[chunkDF['ContestId']==cid]
        numberToRemove = len(cSeriesDF[cSeriesDF['MinutesRemaining']<240])

        X = cSeriesDF['MinutesRemaining'].max() - cSeriesDF['MinutesRemaining'][:-numberToRemove]
        Y = cSeriesDF['Entries'].cumsum()[:-numberToRemove]
        X, Y = list(X), list(Y)   
        
        contest = {'ContestId':cid}
        
        for fFactor in Fs[:]:
            fName = fFactor['label']
            fFactor = fFactor['F']
            A, B = 'TooShort', 'TooShort'
            if(len(X)>=1):
                A, B = processRegr(X, Y, fFactor, SCALED_DATA=True)
            contest[fName+'A'], contest[fName+'B']=A, B    
        results.append(contest)
        contestBar.value+=1
    chunkBar.value+=1


In [28]:
resultsDF = pd.DataFrame(results).set_index('ContestId')

In [29]:
len(resultsDF)

649329

In [31]:
len(resultsDF[resultsDF['f1A']=='TooShort'])

362198

In [32]:
resultsDF.to_csv('data/LR_Values.csv')